# Analysis Ready Data Tutorial Part 2: Use Case 1
## Visualize Images

In [ard_2_use_case_1.ipynb](ard_2_use_case_1.ipynb), we found scenes that match a certain search date and AOI, grouped them by dates, and for each date, we submitted an order that calculated NDVI, clipped the scenes to the AOI, and composited them into one image. In that notebook, we used QGIS to visualize the result. In this notebook, we will visualize the NDVI image with a colormap that makes it easier to understand and we will also mask out any unusable pixels identified in the UDM2.

As mentioned in that notebook, NDVI values range from -1 to 1. Values closer to 1 represent dense, growing vegetation and values closer to 0 represent little to no vegetation.

In this notebook we do a few things to save on memory. If we didn't, the kernel would crash after displaying one or two images. First, we convert the NDVI values from float32 to uint16. This could make a difference in an analytic calculation (although it is wise to keep in mind that NDVI is *normalized*), but our eyes won't be able to tell the difference. Also, we downsample the pixels by just skipping a pixel or two (or 4 or 8...) in each direction. This is pretty blunt-force downsampling, but we don't really need anything fancy for visualization.

#### Import Dependencies

In [ ]:
from itertools import chain
import json
import os
from pathlib import Path

import numpy as np
import rasterio
from zipfile import ZipFile

#### Step 1: Locate order folders

In [ ]:
# you should fill these in with orders you downloaded in the previous notebook. Order IDs will not be the same
order_folders = [
    'data/use_case_1/81b24ca7-a1bf-4e43-9624-20267d233904',
    'data/use_case_1/99e5f1bb-4067-4816-bf85-d5656f496f87'
]

print(order_folders)

In [ ]:
def get_download_locations(download_dir):
    manifest_file = os.path.join(download_dir, 'manifest.json')
    with open(manifest_file, 'r') as src:
        manifest = json.load(src)
        
    locations = [os.path.join(download_dir, f['path'])
                 for f in manifest['files']]
    return locations


locations = []
for i in order_folders:
    location = get_download_locations(i)
    locations.append(location)
    print(locations)

# un-nest our locations object
locations = list(chain.from_iterable(locations))

In [ ]:
locations

#### Step 2: Visualize Images

In this section we will find the image files and their associated UDMs and we will visualize them.

The first band of the UDM2 file is the clear/not-clear band. 0: not-clear, 1: clear.

In [ ]:
# these functions are a little different than what was introduced in the 
# best practices tutorial
# first, we have to change our search a little bit to find the composite image
# second, we convert the ndvi image to int16 to save some memory storage

def get_composite_image_and_udm_files(directory):
    print(directory)
    files = [f for f in get_unzipped_files(directory)]
    
    def find_file(files, filename):
        return next((f for f in files if os.path.basename(f) == filename),
                    None)
    imgfile = find_file(files, 'composite.tif')
    
    if imgfile:
        udmfile = find_file(files, 'composite_udm2.tif')
    else:
        udmfile = None
    
    return str(imgfile), str(udmfile)

def read_ndvi_as_int(img_filename, not_clear):
    """the ndvi band is float32. lets save memory by moving to int16"""
    with rasterio.open(img_filename) as img:
        # ndvi is a single-band image
        ndvi = img.read(1)
        
        # scale to int16
        new_dtype = np.int16
        new_max_value = np.iinfo(new_dtype).max
        new_ndvi = (ndvi * new_max_value).astype(new_dtype)
    return new_ndvi

In [ ]:
# this functionality was introduced in the best practices tutorial

def unzip(filename):
    location = Path(filename)
    
    zipdir = location.parent / location.stem
    with ZipFile(location) as myzip:
        myzip.extractall(zipdir)
    return zipdir

def get_unzipped_files(zipdir):
    filedir = zipdir / 'files'
    filenames = os.listdir(filedir)
    return [filedir / f for f in filenames]


def get_image_and_udm_files(file_paths):
    files = [str(p) for p in file_paths]
    
    # the image files are tiffs and are identified with '_SR_' in the name
    img_id = '_AnalyticMS_SR_'
    imgfiles = [f for f in files
                if f.endswith('.tif') and img_id in f]
    
    # get associated udm files for image files
    # each image has a unique id at the beginning of the name
    imgroots = [str(f).split(img_id)[0] for f in imgfiles]
    
    # the udm files are identified with '_udm2' in the name
    udmfiles = [next(f for f in files if f.startswith(r + '_udm2'))
                for r in imgroots]
    
    return imgfiles, udmfiles


# read UDM2 file
def read_notclear(udm2_filename):
    with rasterio.open(udm2_filename) as img:
        # the first band is the clear/not clear band
        mask=img.read(1)
        not_clear = mask == 0
        return not_clear

# there is an issue where some udms aren't the same size as the images
# to deal with this just cut off any trailing rows/columns
# this isn't ideal as it can result in up to one pixel shift in x or y direction
def crop(img, shape):
    return img[:shape[0], :shape[1]]

def crop_and_mask(img, not_clear):
    # crop image and mask to same size
    img_shape = min(img.shape, not_clear.shape)
    return np.ma.array(crop(img, img_shape),
                       mask=crop(not_clear, img_shape))
    
def downsample_img(img, amt=2):
    return img[::amt, ::amt]

In [ ]:
# we demonstrated visualization in the best practices tutorial
# here, we save space by just importing the functionality
from visual import show_ndvi

In [ ]:
for i in locations:
    zipdir = unzip(i)
    imgfile, udmfile = get_composite_image_and_udm_files(zipdir)
    print(imgfile)
    
    ndvi = read_ndvi_as_int(imgfile, udmfile)
    not_clear = read_notclear(udmfile)
    show_ndvi(downsample_img(crop_and_mask(ndvi, not_clear), 4), figsize=(20,20))

Okay, we got some beautiful NDVI images down! Note the UDM2 masking of pixels outside of the footprint and the occasional errant cloud or two.